In [1]:
import pandas as pd
import numpy as np
import os

os.getcwd()

os.chdir("/Users/amymiao/Documents/UC Berkeley/Fall 8.27-12.16/Applied Computing/final-project-repo-group-four")

In [2]:
df = pd.read_csv('data/clean_processed.csv')

In [4]:
df_1990s= df[(df["Year"]>= 1990) & (df["Year"] < 2000)]
df_1990s.shape

(962, 11)

In [5]:
docs = df_1990s['Lyrics'].tolist()

In [29]:
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import numpy as np
import random

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

stopwords_nltk = stopwords.words('english')
lyrics_1990s_stopwords = stopwords_nltk + [
    'la','lo','lola','du','oh','yeah','la la','lo lo','du du','got','gonna',
    'ooh','get','take','da','know','like','one','said','ron','baby',
    'let','go','come','want','toke','say','day','good','see','make',
    'niggas','nigga','love','ya','choo','na','uh','yuh','hmm','wanna',
    'doo','dat','woo','nah','whoomp','yo','whoo','woo','whatta'
]

vectorizer = CountVectorizer(
    stop_words=lyrics_1990s_stopwords,
    ngram_range=(1, 2)
)

umap_model = UMAP(
    n_neighbors=5,
    n_components=10,
    min_dist=0.0,
    metric="cosine",
    random_state=SEED
)

hdbscan_model = HDBSCAN(
    min_cluster_size=8,   # how many docs per topic (granularity)
    min_samples=2,      # how strict the clustering is
    prediction_data=True,
    cluster_selection_method="leaf"  # leaf gives more granular topics
)

topic_model_1990s = BERTopic(
    embedding_model="all-distilroberta-v1",   # all-distilroberta-v1
    vectorizer_model=vectorizer,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    min_topic_size=8,
    calculate_probabilities=True,
    verbose=True
)


In [30]:
topics_1990s, probs_1990s = topic_model_1990s.fit_transform(docs)

# only 10 big topics
topic_model_1990s.reduce_topics(docs, nr_topics=10)

topic_info_1990s = topic_model_1990s.get_topic_info()
topic_info_1990s

2025-11-19 12:35:18,206 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 31/31 [00:23<00:00,  1.34it/s]
2025-11-19 12:35:43,668 - BERTopic - Embedding - Completed ✓
2025-11-19 12:35:43,669 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-19 12:35:44,567 - BERTopic - Dimensionality - Completed ✓
2025-11-19 12:35:44,568 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-19 12:35:44,609 - BERTopic - Cluster - Completed ✓
2025-11-19 12:35:44,610 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-19 12:35:44,809 - BERTopic - Representation - Completed ✓
2025-11-19 12:35:44,965 - BERTopic - Topic reduction - Reducing number of topics
/opt/anaconda3/envs/python310/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

divide by zero encountered in matmul

/opt/anaconda3/envs/python310/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: 

,Topic,Count,Name,Representation,Representative_Docs
0,-1,266,-1_never_time_cause_man,"[never, time, cause, man, back, ride, away, wa...",[This song goes out to anyone that lost a love...
1,0,224,0_heart_never_time_way,"[heart, never, time, way, cry, back, cause, ho...","[ How could the one I gave my heart to, break ..."
2,1,176,1_give_girl_need_real,"[give, girl, need, real, body, yes, cause, fee...",[ Oh yeah baby You got to make your mind up Ye...
3,2,76,2_way_girl_play_hey,"[way, girl, play, hey, dip, time, put, little,...","[Oh, yeah, oh Shorty swing my way You sure l..."
4,3,68,3_check_wake_back_wake wake,"[check, wake, back, wake wake, jump, kick, cau...","[ Party-people! Yeah, Tag Team music in full e..."
5,4,49,4_shoo_alright_promise_rain,"[shoo, alright, promise, rain, shoo doop, alri...",[ Everyone falls in love sometime Sometimes it...
6,5,47,5_ever_friends_never_bright,"[ever, friends, never, bright, way, bright bri...",[ A few questions that I need to know How you ...
7,6,31,6_crossroads_gangsta_back_kid,"[crossroads, gangsta, back, kid, livin, daddy,...",[ Back in the days when I was young I'm not a ...
8,7,17,7_wild_wild wild_west_wild west,"[wild, wild wild, west, wild west, hello, bad,...","[ Harry Truman, Doris Day, Red China, Johnnie ..."
9,8,8,8_night_sweet_last night_last,"[night, sweet, last night, last, another, fant...","[ Another night, another dream, but always you..."


In [31]:
topic_model_1990s.visualize_documents(docs, topics=topics_1990s)

In [23]:
docs_per_topic = topic_model_1990s.get_representative_docs()
docs_per_topic[7]

[' Harry Truman, Doris Day, Red China, Johnnie Ray South Pacific, Walter Winchell, Joe DiMaggio Joe McCarthy, Richard Nixon, Studebaker, Television North Korea, South Korea, Marilyn Monroe Rosenbergs, H-Bomb, Sugar Ray, Panmunjom Brando, The King And I, and The Catcher In The Rye, Eisenhower, Vaccine, England\'s got a new queen Marciano, Liberace, Santayana goodbye   We didn\'t start the fire It was always burning since the world\'s been turning We didn\'t start the fire No, we didn\'t light it, but we tried to fight it   Joseph Stalin, Malenkov, Nasser and Prokofiev Rockefeller, Campanella, Communist Bloc Roy Cohn, Juan Peron, Toscanini, Dacron Dien Bien Phu Falls, "Rock Around the Clock" Einstein, James Dean, Brooklyn\'s got a winning team Davy Crockett, Peter Pan, Elvis Presley, Disneyland Bardot, Budapest, Alabama, Khrushchev Princess Grace, Peyton Place, Trouble in the Suez   We didn\'t start the fire It was always burning since the world\'s been turning We didn\'t start the fire 